In [73]:
#test_gender.py
import os
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio,sr, 0.025, 0.01, 26 ,nfft=2048,winfunc=np.hamming,appendEnergy = False)
    feat     = np.asarray(())
    for i in range(features.shape[0]):
        temp = features[i,:]
        if np.isnan(np.min(temp)):
            continue
        else:
            if feat.size == 0:
                feat = temp
            else:
                feat = np.vstack((feat, temp))
    features = feat;
    features = preprocessing.scale(features)
    return features

#path to testing data
sourcepath = "pygender/test_data/AudioSet/clips"      
#path to saved models    
modelpath  = "pygender"   

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]
print(gmm_files)
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
genders   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]
files     = [os.path.join(sourcepath,f) for f in os.listdir(sourcepath) 
              if f.endswith(".wav")] 

for f in files:
    print(f.split("/")[-1])
    sr, audio  = read(f)
    features   = get_MFCC(sr,audio)
    scores     = None
    log_likelihood = np.zeros(len(models))
    for i in range(len(models)):
        gmm    = models[i]         #checking with each model one by one
        scores = np.array(gmm.score(features))
        log_likelihood[i] = scores.sum()
    winner = np.argmax(log_likelihood)
    print("\tdetected as - ", genders[winner],"\n\tscores:ball ",log_likelihood[0],",pen ", log_likelihood[1],",dol ", log_likelihood[2],",illa ", log_likelihood[3],",iLLAa ", log_likelihood[4],"\n")
    
    
    
    
    

['pygender/ball.gmm', 'pygender/pen.gmm', 'pygender/iLLAa.gmm', 'pygender/illa.gmm', 'pygender/doll.gmm']
record-24-03-2019_19-07-38.wav
	detected as -  ball 
	scores:ball  -38.50363869205408 ,pen  -41.42042747083092 ,dol  -38.762056428894425 ,illa  -39.816947383582956 ,iLLAa  -39.68028005431532 

record-24-03-2019_19-07-10.wav
	detected as -  ball 
	scores:ball  -35.500452096861046 ,pen  -39.20464486719333 ,dol  -38.11989148899528 ,illa  -38.3193955186598 ,iLLAa  -38.97025532004234 

pen 100.wav
	detected as -  pen 
	scores:ball  -42.825814545283464 ,pen  -36.54811243437755 ,dol  -37.64801558462632 ,illa  -39.393598955700625 ,iLLAa  -42.04534722289606 

illa male 2.wav
	detected as -  iLLAa 
	scores:ball  -45.57742604117382 ,pen  -46.74706834416381 ,dol  -38.91450606097618 ,illa  -39.824550231682906 ,iLLAa  -46.62235078568235 

doll 100.wav
	detected as -  pen 
	scores:ball  -38.8340194208953 ,pen  -36.913763847092135 ,dol  -37.490196866843405 ,illa  -39.09192771433276 ,iLLAa  -38.578